<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/spectrum_v2-1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Enviroment

## Loading Drive and folders

In [255]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [256]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

## Loading Necesary Libraries

In [257]:
# Importing necesary libraries
import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
import seaborn as sn
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

# Loading Data

In [258]:
data_path=path+"/Spectrum"

qpsk_path="/QPSK/A.10hops100km/"
qam_path="/16QAM/A.10hops100km"

#file_name="/dataSet_Spectrum_Samples_QPSK_37GHz_LongHaul_output_10x100.xlsx"
file_name = "/QPSK_dummy.xlsx"
# Skipping first row since only contain "Frequency" headers
df_qpsk = pd.read_excel(data_path+qpsk_path+file_name, sheet_name = "Sheet1", skiprows=1)

#file_name="/dataSet_Spectrum_Samples_16QAM_75GHz_LongHaul_output_10x100.xlsx"
file_name = "/16QAM_dummy.xlsx"
df_qam = pd.read_excel(data_path+qam_path+file_name, sheet_name = "Sheet1", skiprows=1)

#Diff in column 5 to 1,286 0.03125

In [259]:
#dump_file = dict desireed to be saved
#filename = just the name of the file (file saved on "pickle_folder")

def pickle_out(dump_file, filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  pickleout = open(filepath, "wb")
  pickle.dump(dump_file, pickleout)
  pickleout.close()

def pickle_in(filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  picklein = open(filepath, "rb")
  loaded_dict = pickle.load(picklein)
  return loaded_dict

# Aggregator

In [260]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
    i+=1
  return prime_nums

In [261]:
def aggreagator_selection(stop_gap=25, total_num_of_data=1281, min_prime = False):
  current_freq_offset = 0.03125
  
  # Find the prime numbers where the total_num_of_data is divisible
  prime_nums = prime_number_finder(stop_gap, total_num_of_data)
  
  # Find the offsets of the new aggregators
  new_freq_offsets = []
  for nums in prime_nums:
    freq_offset = current_freq_offset*(nums-1)
    new_freq_offsets.append(freq_offset)

  # Select the prime number we would use to aggregation
  if min_prime == False:
    # Ask the user for the desired aggregator
    print('Please select one of the following prime number to conduct the aggregation {} with frequencies offsets of {}(PHz) respectively. (Current Freq offset is 0.03125 PHz)'.format(prime_nums, new_freq_offsets))
    while True:
      user_selection = input('Prime number to conduct the aggregation? ')
      if int(user_selection) not in prime_nums:
        print('Please select one of the available prime numbers. (Current selection is not avaliable)')
      else:
        break
    return int(user_selection)
  else:
    user_selection = min(prime_nums)
    freq_selected = min(new_freq_offsets)
    print('Selected the least common denominator for aggregation ({}) with a frequency of {}.'.format(user_selection, freq_selected))
    return int(user_selection)

In [262]:
def aggreagator(df, stop_gap=25, total_num_of_data=1281, min_prime = False):
  agg_num = aggreagator_selection(stop_gap, total_num_of_data, min_prime = min_prime)
  
  # Creating new df for aggregate values
  agg_df = pd.DataFrame()
  agg_df = df[df.columns[0:5]]

  agg=0
  loop_cnt=0
  init_column_cnt = 5
  while agg<=total_num_of_data:
    loop_cnt+=1
    # Obtain current last columns stop
    agg=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg]]

    init_column_cnt += (agg_num)
    #print(init_column_cnt)
    # Get average of freq values for new header
    headers = list(new_df.columns.values)
    headers_array = np.asarray(headers)
    new_header_name = np.mean(headers_array)

    # Add aggragated values to new df
    agg_df[str(new_header_name)] = new_df.mean(axis = 1)
  agg_dict = agg_df.to_dict()
  return agg_df, agg_dict


In [263]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
    i+=1
  return prime_nums

def aggreagator_selection(stop_gap=25, total_num_of_data=1281, min_prime = False):
  current_freq_offset = 0.03125
  
  # Find the prime numbers where the total_num_of_data is divisible
  prime_nums = prime_number_finder(stop_gap, total_num_of_data)
  
  # Find the offsets of the new aggregators
  new_freq_offsets = []
  for nums in prime_nums:
    freq_offset = current_freq_offset*(nums-1)
    new_freq_offsets.append(freq_offset)

  if min_prime == False:
    # Ask the user for the desired aggregator
    print('Please select one of the following prime number to conduct the aggregation {} with frequencies offsets of {}(PHz) respectively. (Current Freq offset is 0.03125 PHz)'.format(prime_nums, new_freq_offsets))
    while True:
      user_selection = input('Prime number to conduct the aggregation? ')
      if int(user_selection) not in prime_nums:
        print('Please select one of the available prime numbers. (Current selection is not avaliable)')
      else:
        break
    return int(user_selection)
  else:
    user_selection = min(prime_nums)
    freq_selected = min(new_freq_offsets)
    print('Selected the least common denominator for aggregation ({}) with a frequency of {}.'.format(user_selection, freq_selected))
    return int(user_selection)

def aggreagator(df, stop_gap=25, total_num_of_data=1281, min_prime = True):
  agg_num = aggreagator_selection(stop_gap, total_num_of_data, min_prime = min_prime)
  
  # Creating new df of aggregate values
  agg_df = pd.DataFrame()
  agg_df = df[df.columns[0:5]]

  agg_column=0
  loop_cnt=0
  init_column_cnt = 5
  while agg_column<=total_num_of_data:

    # Obtain current last column to stop the range aggregation
    agg_column=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg_column]]

    # Get average of freq values for new header
    headers = list(new_df.columns.values)
    headers_array = np.asarray(headers)
    new_header_name = np.mean(headers_array)

    # Add aggragated values to new df
    agg_df[str(new_header_name)] = new_df.mean(axis = 1)

    init_column_cnt += (agg_num)
    loop_cnt+=1

  #Create dict of agg df
  agg_dict = agg_df.to_dict()
  return agg_df, agg_dict

In [264]:
# QPSK AGG
QPSK_agg_df, QPSK_agg_dict = aggreagator(df_qpsk, stop_gap=100, total_num_of_data=(df_qpsk.shape[1]-5), min_prime = True)
#pickle_out(QPSK_agg_dict, "test_agg_spectrum_QPSK_"+str(stop))

Selected the least common denominator for aggregation (3) with a frequency of 0.0625.


# Get model

In [441]:
def get_dummy_spectrum(df):
  dummy_df = df.groupby(['#node', 'power_dBm']).mean()
  dummy_df.drop(['sampleid', 'Channels'], axis = 1, inplace = True)
  dummy_dict = dummy_df.to_dict()
  
  return dummy_df, dummy_dict
  #Return a tuple of (node, power) as keys in the nested dict and indexes

In [442]:
# Dummy Model QPSK
QPSK_dummy_df, QPSK_dummy_dict = get_dummy_spectrum(df_qpsk)

#pickle_out(QPSK_dummy_dict, "dummy_AGG_spectrum_QPSK")

In [443]:
QPSK_dummy_df.head()

,,Distance_km,-20,-19.96875,-19.9375,-19.90625,-19.875,-19.84375,-19.8125,-19.78125,-19.75,...,19.71875,19.75,19.78125,19.8125,19.84375,19.875,19.90625,19.9375,19.96875,20
#node,power_dBm,,,,,,,,,,,,,,,,,,,,,
1,-2,0.0,-52.955056,-54.126681,-52.713840,-54.542630,-53.699036,-52.716651,-54.298754,-52.256947,-56.682823,...,-55.905776,-54.248709,-55.992318,-53.838975,-50.924380,-53.185248,-53.206559,-54.780695,-52.482275,-53.598605
2,-2,100.0,-51.813386,-53.194155,-52.697528,-52.418356,-52.486649,-50.289808,-54.422782,-54.110832,-55.149438,...,-53.005446,-55.507125,-54.494074,-50.651776,-54.789647,-53.624259,-51.423731,-55.126913,-57.266619,-51.729962
3,-2,200.0,-55.626777,-54.634329,-54.702958,-56.522545,-50.522361,-51.649987,-51.647787,-57.084303,-53.951639,...,-51.567208,-56.796925,-51.015703,-52.421725,-54.815529,-51.357553,-51.877358,-57.233009,-56.753557,-53.614796


# Get Data (Node and Power)

In [322]:
# available filters are -2, -1, 0

def get_data_spectrum(df, node, power):
  #data_df = pd.DataFrame({'#node': [int(node)], 'power_dBm': [int(power)]})
  # Get selected node
  data_df = df[df['#node'].isin([int(node)])]

  # Get selected power
  data_df = data_df[data_df['power_dBm'].isin([int(power)])]

  data_dict = data_df.to_dict()
  
  return data_df, data_dict
  #Return a tuple of (node, power) as keys in the nested dict and indexes

In [323]:
data_df, data_dict = get_data_spectrum(df_qpsk, node=1, power=-2)

In [324]:
data_df.head()

,sampleid,Channels,Distance_km,power_dBm,#node,-20,-19.96875,-19.9375,-19.90625,-19.875,...,19.71875,19.75,19.78125,19.8125,19.84375,19.875,19.90625,19.9375,19.96875,20
0,1,1,0,-2,1,-50.396272,-51.195507,-52.325501,-57.685476,-51.730270,...,-50.701297,-50.425586,-57.242411,-54.978886,-58.984450,-52.289897,-50.007652,-56.562352,-50.661037,-57.857937
1,2,1,0,-2,1,-49.464198,-62.300068,-49.878752,-45.576386,-50.935177,...,-52.919979,-61.894104,-60.851791,-53.621297,-47.427855,-57.669031,-51.735427,-56.145979,-51.259631,-46.965564
2,3,1,0,-2,1,-52.100342,-54.825356,-49.933739,-53.418630,-55.604328,...,-52.437768,-48.572056,-58.180955,-50.003337,-49.935404,-52.854423,-52.839831,-56.938673,-48.511327,-53.268586
3,4,1,0,-2,1,-61.429210,-54.973444,-45.026184,-63.731247,-45.706566,...,-56.792207,-59.099399,-45.632244,-56.454966,-46.126678,-50.632960,-48.675022,-50.899182,-64.439508,-46.552580
4,5,1,0,-2,1,-51.777530,-57.344578,-55.337847,-52.396041,-53.875442,...,-48.008932,-51.073828,-49.675923,-57.762095,-49.534186,-59.334809,-59.237640,-62.395677,-46.607586,-69.244091


# Remove outliers

## Sample outliears remove

In [325]:
def remove_outliers(df, row_index, lower_qr=25, upper_qr=75):

  # IQR
  Q1 = np.percentile(df.iloc[[row_index]], lower_qr, interpolation = 'midpoint')
  
  Q3 = np.percentile(df.iloc[[row_index]], upper_qr, interpolation = 'midpoint')
  IQR = Q3 - Q1
  
  # Above Upper bound
  upper = df.iloc[[row_index]] >= (Q3+1.5*IQR)
  upper_bound_drop_cols = np.where(upper)[1]
  
  # Below Lower bound
  lower = df.iloc[[row_index]] <= (Q1-1.5*IQR)
  lower_bound_drop_cols = np.where(lower)[1]
  
  
  drop_cols = np.concatenate((upper_bound_drop_cols, lower_bound_drop_cols), axis=None)

  new_df = df.drop(df.columns[drop_cols], axis=1)
  
  new_sample = new_df.iloc[[row_index]]
  return new_sample, drop_cols

In [326]:
df_qpsk_no_outliers, drop_cols = remove_outliers(data_df, row_index=0, lower_qr=25, upper_qr=75)

In [327]:
df_qpsk_no_outliers.shape

(1, 1082)

In [328]:
df_qpsk_no_outliers.head()

,sampleid,Channels,Distance_km,power_dBm,#node,-16.875,-16.84375,-16.8125,-16.78125,-16.75,...,16.53125,16.5625,16.59375,16.625,16.65625,16.6875,16.71875,16.75,16.78125,16.875
0,1,1,0,-2,1,-36.637486,-32.441265,-30.236894,-35.012474,-37.655658,...,-32.18383,-25.710114,-32.996736,-27.928139,-32.138923,-29.580422,-35.15666,-38.609825,-35.112885,-37.250151


## Model match new sample df

In [339]:
def model_match_sample(model_df, sample_drop_cols):

  new_drop_cols = []
  for col in sample_drop_cols:
    new_drop_cols.append(int(col-4))

  new_model_df = model_df.drop(model_df.columns[new_drop_cols], axis=1)
  return new_model_df

In [451]:
QPSK_dummy_df_no_outliers = model_match_sample(QPSK_dummy_df, drop_cols)

In [386]:
display(QPSK_dummy_df_no_outliers)

,,Distance_km,-16.875,-16.84375,-16.8125,-16.78125,-16.75,-16.71875,-16.6875,-16.65625,-16.625,...,16.53125,16.5625,16.59375,16.625,16.65625,16.6875,16.71875,16.75,16.78125,16.875
#node,power_dBm,,,,,,,,,,,,,,,,,,,,,
1,-2,0.0,-43.197842,-38.957745,-35.590416,-35.420975,-32.973116,-32.417927,-31.202355,-28.279406,-30.650614,...,-30.051103,-27.717405,-29.006557,-28.084133,-29.470861,-35.134268,-32.931001,-33.604595,-32.276210,-41.197859
2,-2,100.0,-47.986502,-45.360884,-42.007214,-42.039656,-37.850284,-38.685132,-37.009589,-34.271041,-35.963138,...,-36.951914,-33.703721,-36.091838,-34.081474,-35.679383,-41.595027,-39.126932,-39.861682,-38.568821,-44.944971
3,-2,200.0,-48.742821,-49.014173,-44.829997,-46.838368,-44.407271,-42.853775,-41.536193,-40.579536,-42.571873,...,-42.437070,-38.895391,-43.137975,-41.639048,-41.284961,-44.696975,-44.471855,-45.539585,-45.475736,-49.776780


In [452]:
def match_model_with_sample(model_df):
  model_df.insert(loc = 0, column = 'sampleid', value = 1)
  model_df.insert(loc = 1, column = 'Channels', value = 1)
  model_df.insert(loc = 3, column = 'power_dBm', value = 0)
  model_df.insert(loc = 4, column = '#node', value = 0)  
  
  row = 0
  for idx in model_df.index:
    model_df.iloc[row, [3]] = idx[1]
    model_df.iloc[row, [4]] = idx[0]
    row += 1

  return model_df

In [453]:
match_QPSK_dummy_df_no_outliers = match_model_with_sample(QPSK_dummy_df_no_outliers)

In [454]:
display(match_QPSK_dummy_df_no_outliers)

,,sampleid,Channels,Distance_km,power_dBm,#node,-16.875,-16.84375,-16.8125,-16.78125,-16.75,...,16.53125,16.5625,16.59375,16.625,16.65625,16.6875,16.71875,16.75,16.78125,16.875
#node,power_dBm,,,,,,,,,,,,,,,,,,,,,
1,-2,1,1,0.0,-2,1,-43.197842,-38.957745,-35.590416,-35.420975,-32.973116,...,-30.051103,-27.717405,-29.006557,-28.084133,-29.470861,-35.134268,-32.931001,-33.604595,-32.276210,-41.197859
2,-2,1,1,100.0,-2,2,-47.986502,-45.360884,-42.007214,-42.039656,-37.850284,...,-36.951914,-33.703721,-36.091838,-34.081474,-35.679383,-41.595027,-39.126932,-39.861682,-38.568821,-44.944971
3,-2,1,1,200.0,-2,3,-48.742821,-49.014173,-44.829997,-46.838368,-44.407271,...,-42.437070,-38.895391,-43.137975,-41.639048,-41.284961,-44.696975,-44.471855,-45.539585,-45.475736,-49.776780


In [455]:
print(data_df.shape)
print(df_qpsk_no_outliers.shape)
print(QPSK_dummy_df.shape)
print(QPSK_dummy_df_no_outliers.shape)
print(match_QPSK_dummy_df_no_outliers.shape)

(10, 1286)
(1, 1082)
(3, 1282)
(3, 1082)
(3, 1082)


# Separate tails

In [163]:
def tails_generator(row_df):
  total_columns = row_df.shape[1]-5
  mid_point = int(total_columns/2)

  new_sample = row_df.iloc[:, 0:5]
  mid_point = mid_point+5

  left_tail = row_df.iloc[:, 5:mid_point]
  rigth_tail = row_df.iloc[:, mid_point+1:row_df.shape[1]]

  return left_tail, rigth_tail


In [164]:
left_tail, rigth_tail = tails_generator(df_qpsk_no_outliers)

In [167]:
print(left_tail.shape)
print(rigth_tail.shape)

(1, 517)
(1, 516)


# Linear Regresion

In [ ]:
def obtain_regresion_features(sample_df, model_df):

  # Get model data
  model_df = model_df.iloc[0].to_numpy()
  model_df = model_df[5:]
  y = np.array(model_df)
  features_dict = {}
  # Get Sample data
  for row in range(sample_df.shape[0]):
    features = []

    row_data_sample = sample_df.iloc[row].to_numpy()
    row_data_sample = row_data_sample[5:]
    x = np.array(row_data_sample).reshape((-1, 1))
    
    # Get linear regresion features
    model = LinearRegression()
    model.fit(x, y)
    model = LinearRegression().fit(x, y)
    r_sq = model.score(x, y)
    intercept = model.intercept_
    slope = model.coef_
    
    features.append(r_sq)
    features.append(intercept)
    features.append(slope[0])

    features_dict['Sample_'+str(row+1)] = features
  # Return dictionary with sample as key and a array with (r_sq, intercept, slope) as values
  return features_dict

In [ ]:
def linear_regresion(node_df_left, node_df_right, model_node_row_left, model_node_row_right):

  # Checking left side
  left_features_dict = obtain_regresion_features(node_df_left, model_node_row_left)

  # Checking right side
  right_features_dict = obtain_regresion_features(node_df_right, model_node_row_right)
  
  r_sqr_agg = []
  intercept_agg = []
  slope_agg = []
  combined_features_dict = {}
  for k, val_array in left_features_dict.items():
    combined_features = []
    # Combining r_sqr
    combined_r_sq = (val_array[0] + right_features_dict[k][0])/2 
    combined_features.append(combined_r_sq)
    r_sqr_agg.append(combined_r_sq)

    # Combining intercept
    combined_intercept = (val_array[1] + right_features_dict[k][1])/2 
    combined_features.append(combined_intercept)
    intercept_agg.append(combined_intercept)

    # Combining slope
    combined_slope = (val_array[2] + right_features_dict[k][2])/2 
    combined_features.append(combined_slope)
    slope_agg.append(combined_slope)

    combined_features_dict[k] = combined_features

  # Get overall average of each feature
  r_sqr_val = np.average(r_sqr_agg)
  intercept_val = np.average(intercept_agg)
  slope_val = np.average(slope_agg)
  # Return dictionary with sample as key and a array with (r_sq, intercept, slope) as values with combined left and right curves
  return combined_features_dict, r_sqr_val, intercept_val, slope_val
  
  # Prediction
  #y_pred = model.predict(x)
  #print('predicted response:', y_pred, sep='\n')

In [ ]:
def linear_regresion_v2(node_df_left, node_df_right, model_node_row_left, model_node_row_right):

  # Checking left side
  left_features_dict = obtain_regresion_features(node_df_left, model_node_row_left)

  # Checking right side
  right_features_dict = obtain_regresion_features(node_df_right, model_node_row_right)
  
  r_sqr_agg = []

  for k, val_array in left_features_dict.items():
    # Combining r_sqr
    combined_r_sq = (val_array[0] + right_features_dict[k][0])/2 
    r_sqr_agg.append(combined_r_sq)

  # Get overall average of each feature
  r_sqr_val = np.average(r_sqr_agg)

  # Return dictionary with sample as key and a array with (r_sq, intercept, slope) as values with combined left and right curves
  return r_sqr_val